Co to są *pipelines* w sci-kit learn i jak je wykorzystać?
Czyli **bardziej efektywne szukanie najlepszego modelu**.

Jeśli zajmujesz się tworzeniem modeli na przykład klasyfikujących jakieś dane to pewnie wielokrotnie powtarzasz te same kroki:

* wczytanie danych
* oczyszczenie danych
* uzupełnienie braków
* przygotowanie dodatkowych cech (*feature engineering*)
* podział danych na treningowe i testowe
* dobór hyperparametrów modelu
* trenowanie modelu
* testowanie modelu (sprawdzenie skuteczności działania)

I tak w kółko, z każdym nowy modelem.

Jest to dość nudne i dość powtarzalne. Szczególnie jak trzeba wykonać różne kroki transformacji danych w różnej kolejności - *upierdliwe* staje się zmienianie kolejności w kodzie.

Dlatego wymyślono **pipelines**.

Dlatego w tym i kolejnych postach zajmiemy się tym mechanizmem.

Zaczniemy od podstaw *data science*, czyli...

In [1]:
# bez tego nie ma data science! ;)
import pandas as pd

# być może coś narysujemy
import matplotlib.pyplot as plt
import seaborn as sns

import time

Wszystkie elementy jakich będziemy używać znajdują się w ramach biblioteki **scikit-learn**. Zaimportujemy co trzeba plus kilka modeli z oddzielnych bibliotek.

In [14]:
from sklearn.model_selection import train_test_split

# modele
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# preprocessing
## zmienne ciągłe
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
## zmienne kategoryczne
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


# dodatkowe modele spoza sklearn
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

Skąd wziąć dane? Można użyć wbudowanych w sklearna *irysów* czy *boston housing* ale mi zależało na znalezieniu takiego datasetu, który będzie zawierał cechy zarówno ciągłe jak i kategoryczne. Takim zestawem jest **Adult** znany też jako **Census Income**, a ściągnąć go można z [UCI](https://archive.ics.uci.edu/ml/datasets/Adult) (studenci i absolwenci AGH mogą mylić z [UCI](https://historia.agh.edu.pl/wiki/Uczelniane_Centrum_Informatyki) w budynku C-1).

Pobieramy (potrzebne nam będą pliki [adult.data](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data) oraz [adult.test](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test)) dane, wrzucamy surowe pliki do katalogu *data/* i wczytujemy.

In [3]:
# dane nie mają nagłówka - samo sobie nadamy nazwy kolumn
col_names= ['age', 'work_class', 'final_weight', 'education', 'education_num',
            'marital_status', 'occupation', 'relationship', 'race', 'sex',
            'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
            'year_income']

# wczytujemy dane
adult_dataset = pd.read_csv("data/adult.data",
                            engine='python', sep=', ', # tu jest przeciek i spacja!
                            header=None, names=col_names,
                            na_values="?")

# kolumna 'final_weight' do niczego się nie przyda, więc od razu ją usuwamy
# wiadomo to z EDA, które tutaj pomijamy
adult_dataset.drop('final_weight', axis=1, inplace=True)

# usuwamy braki, żeby uprościć przykład
adult_dataset.dropna(inplace=True)

Zobaczmy jakie mamy typy danych w kolumnach:

In [4]:
adult_dataset.dtypes

age                int64
work_class        object
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
year_income       object
dtype: object

A teraz standardowo - dzielimy dane na zbiór treningowy i testowy. Przy okazji z całej ramki danych wyciągamy kolumnę `year_income` jako **Y**, a resztę jako **X**. Szalenie wygodnym jest nazywanie cech zmienną **X** a *targetów* **y** - przy *Ctrl-C + Ctrl-V* ze StackOverflow niczego właściwie nie trzeba robić ;)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(adult_dataset.drop('year_income', axis=1),
                                                    adult_dataset['year_income'],
                                                    test_size=0.3,
                                                    random_state=42)

Sporą część wstępnej analizy pomijam w tym wpisie, ale jeśli nie wiesz dlaczego wybieram takie a nie inne kolumny to zrób samodzielnie analizę danych w tym zbiorze. 

**Czas na trochę informacji o *rurociągach*.**

W dużym uproszszczeniu są to połączone sekwencyjnie (jedna za drugą, wyjście pierwszej trafia na wejście drugiej i tak dalej do końca) operacje. Operacje czyli klasy, które posiadają metody `.fit()` i `.transform()`.

Rurociąg może składać się z kilku rurociągów połączonych jeden za drugim - taki model zastosujemy za chwilę.

Powiedzieliśmy sobie wyżej, że pierwszy krok to przygotowanie danych - odpowiednie transformacje danych źródłowych i ewentualnie uzupełnienie danych brakujących. W dzisiejszym przkładzie brakujące dane (około 7% całości) po prostu wyrzuciliśmy przez `dropna()`, więc uzupełnieniem braków się nie zajmujemy. Ale może w kolejnej części już tak.

Drugim krokiem jest przesłanie danych odpowiednio obrobionych do modelu i jego wytrenowanie.

**No to do dzieła, już konkretnie - transformacja danych**

Zatem na początek przygotujemy sobie fragmenty całego *rurociągu* odpowiedzialnego za transformacje kolumn. Mamy dwa typy kolumn, zatem zbudujemy dwa małe rurociągi.

Pierwszy będzie odpowiedzialny za kolumny z wartościami liczbowymi. Nie wiemy czy są to wartości ciągłe (jak na przykład wiek) czy dyskretne (tutaj taką kolumną jest `education_num` mówiąca o poziome edukacji) i poniżej bierzemy wszystkie jak leci. Znowu: porządna EDA wskaże nam odpowiednie kolumny.

Najpierw wybieramy wszystkie kolumny o typie numerycznym, a potem budujemy mini-rurociąg `transformer_numerical`, którego jedynym krokiem będzie wywołanie `StandardScaler()` zapisane pod nazwą `num_trans` (to musi być unikalne w całym procesie). Kolejny krok łatwo dodać - po prostu dodajemy kolejnego *tupla* w takim samym schemacie.

**Co nam to daje?** Ano daje to tyle, że mamy konkretną nazwę dla konkretnego kroku. Później możemy się do niej dostać i na przykład zmienić: zarówno metodę wywoływaną w tym konkretnym kroku jak i parametry tej metody.

In [6]:
# lista kolumn numerycznych
cols_numerical = X_train.select_dtypes(include=['int64', 'float64']).columns

# transformer dla kolumn numerycznych
transformer_numerical = Pipeline(steps = [
    ('num_trans', StandardScaler())
])

To samo robimy dla kolumn z wartościami kategorycznymi - budujemy mini-rurociąg `transformer_categorical`, który w kroku `cat_trans` wywołuje `OneHotEncoder()`.

In [7]:
# lista kolmn kategorycznych
cols_categorical = ['work_class', 'education', 'marital_status', 'occupation',
                    'relationship', 'race', 'sex', 'native_country']

# transformer dla kolumn numerycznych
transformer_categorical = Pipeline(steps = [
    ('cat_trans', OneHotEncoder())
])

Co dalej? Z tych dwóch małych rurociągów zbudujemy większy - `preprocessor`. Właściwie to będzie to swego rodzaju rozgałęzienie - **ColumnTransformer** który jedne kolumny puści jednym mini-rurociągiem, a drugie - drugim. I znowu: tutaj może być kilka elementów, oddzielne *przepływy* dla konkretnych kolumn (bo może jedne ciągłe chcemy skalować w jeden sposób, a inne w inny? A może jedne zmienne chcemy uzupełnić średnią a inne medianą?) - mamy pełną swobodę.

In [8]:
# preprocesor danych
preprocessor = ColumnTransformer(transformers = [
    ('numerical', transformer_numerical, cols_numerical),
    ('categorical', transformer_categorical, cols_categorical)
])

Cała rura to złożenie odpowiednich elementów w całość - robiliśmy to już wyżej:

In [24]:
pipe = Pipeline(steps = [
                ('preprocessor', preprocessor),
                ('classifier', RandomForestClassifier())
            ])

Teraz **cały proces** wygląda następująco:

* najpierw preprocessing:
    * dla kolumn liczbowych wykonywany jest `StandardScaler()`
    * dla kolumn kategorycznych - `OneHotEncoder()`
* złożone dane przekazywane są do `RandomForestClassifier()`

 
 Proces trenuje się dokładne tak samo jak model - poprzez wywołanie metody `.fit()`:

In [25]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical',
                                                  Pipeline(memory=None,
                                                           steps=[('num_trans',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  Index(['age', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_wee...
                                          

Oczywiście predykcja działa tak samo *jak zawsze*:

In [26]:
pipe.predict(X_test)

array(['<=50K', '<=50K', '<=50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

Są też metody zwracające prawdopodobieństwo przypisania do każdej z klas `.predict_proba()` oraz jego logarytn `.predict_log_proba()`.

Po wytrenowaniu na danych treningowych (cechy *X_train*, target *y_train*) możemy zobaczyć ocenę modelu na danych testowych (odpowiednio *X_test* i *y_test*):

In [27]:
pipe.score(X_test, y_test)

0.850259697204111

Świetnie, świetne, ale to samo można bez tych pipelinów, nie raz na Kaggle tak robili i działało. Więc **po co to wszystko?**

Ano po to, co nastąpi za chwilę.

Mamy cały proces, każdy jego krok ma swoją nazwę, prawda? A może zamiast *StandardScaler()* lepszy będzie *MinMaxScaler()*? A może inna klasa modeli (zamiast lasów losowych np. XGBoost?). A gdyby sprawdzić każdy model z każdą transformacją? No to się robi sporo kodu... A nazwane kroki w procesie pozwalają na prostą podmiankę!

Zdefiniujmy sobie przestrzeń poszukiwań najlepszzego modelu i najlepszych transformacji:

In [28]:
# klasyfikatory                            
classifiers = [
    DummyClassifier(strategy='stratified'),
    LogisticRegression(max_iter=500), # można tutaj podać hiperparametry
    KNeighborsClassifier(2), # 2 bo mamy dwie klasy
    ExtraTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    XGBClassifier(),
    CatBoostClassifier(silent=True),
    LGBMClassifier(verbose=-1)
]

# transformatory dla kolumn liczbowych
scalers = [StandardScaler(), MinMaxScaler(), Normalizer()]

# transformatory dla kolumn kategorycznych
cat_transformers = [OrdinalEncoder(), OneHotEncoder()]

Teraz w zagnieżdżonych pętlach możemy sprawdzić *każdy z każdym* podmieniając klasyfikatory i transformatory (cała pętla trochę się kręci):

In [42]:
# miejsce na zebranie wyników
models_df = pd.DataFrame()

# przygotowujemy pipeline
pipe = Pipeline(steps = [
    ('preprocessor', preprocessor), # mniejszy pipeline
    ('classifier', None) # to ustalimy za moment
])

# dla każdego typu modelu zmieniamy kolejne transformatory kolumn
for model in classifiers:
    for num_tr in scalers:
        for cat_tr in cat_transformers:
            # odpowiednio zmieniamy jego paramety - dobieramy transformatory
            pipe_params = {
                'preprocessor__numerical__num_trans': num_tr,
                'preprocessor__categorical__cat_trans': cat_tr,
                'classifier': model
            }
            pipe.set_params(**pipe_params)

            # zbieramy w dict parametry dla Pipeline
            param_dict = {
                        'model': model.__class__.__name__,
                        'num_trans': num_tr.__class__.__name__,
                        'cat_trans': cat_tr.__class__.__name__
                    }

            # wypisujemy je
            print('='*40)
            print(param_dict)
            
            # trenujemy tak przygotowany model (cały pipeline) mierząc ile to trwa
            start_time = time.time()
            pipe.fit(X_train, y_train)   
            end_time = time.time()

            # sprawdzamy jak wyszło
            score = pipe.score(X_test, y_test)

            # i informyjemy o tym
            print(f"Score: {round(100*score, 2)}%, Time: {(end_time-start_time):.3} s")

            # i dodajemy do tabelki wszystkich wyników (razem z parametrami)
            param_dict['score'] = score
            param_dict['time_elapsed'] = end_time - start_time
            
            models_df = models_df.append(pd.DataFrame(param_dict, index=[0]))

{'model': 'DummyClassifier', 'num_trans': 'StandardScaler', 'cat_trans': 'OrdinalEncoder'}
Score: 62.29%, Time: 0.1059 s
{'model': 'DummyClassifier', 'num_trans': 'StandardScaler', 'cat_trans': 'OneHotEncoder'}
Score: 62.52%, Time: 0.1041 s
{'model': 'DummyClassifier', 'num_trans': 'MinMaxScaler', 'cat_trans': 'OrdinalEncoder'}
Score: 62.46%, Time: 0.1132 s
{'model': 'DummyClassifier', 'num_trans': 'MinMaxScaler', 'cat_trans': 'OneHotEncoder'}
Score: 62.8%, Time: 0.1334 s
{'model': 'DummyClassifier', 'num_trans': 'Normalizer', 'cat_trans': 'OrdinalEncoder'}
Score: 62.55%, Time: 0.08864 s
{'model': 'DummyClassifier', 'num_trans': 'Normalizer', 'cat_trans': 'OneHotEncoder'}
Score: 62.47%, Time: 0.1308 s
{'model': 'LogisticRegression', 'num_trans': 'StandardScaler', 'cat_trans': 'OrdinalEncoder'}
Score: 81.79%, Time: 2.772 s
{'model': 'LogisticRegression', 'num_trans': 'StandardScaler', 'cat_trans': 'OneHotEncoder'}
Score: 85.03%, Time: 1.526 s
{'model': 'LogisticRegression', 'num_trans':

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Score: 81.63%, Time: 9.622 s
{'model': 'LogisticRegression', 'num_trans': 'MinMaxScaler', 'cat_trans': 'OneHotEncoder'}
Score: 84.86%, Time: 2.194 s
{'model': 'LogisticRegression', 'num_trans': 'Normalizer', 'cat_trans': 'OrdinalEncoder'}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Score: 78.35%, Time: 8.648 s
{'model': 'LogisticRegression', 'num_trans': 'Normalizer', 'cat_trans': 'OneHotEncoder'}
Score: 84.21%, Time: 1.529 s
{'model': 'KNeighborsClassifier', 'num_trans': 'StandardScaler', 'cat_trans': 'OrdinalEncoder'}
Score: 81.04%, Time: 0.8799 s
{'model': 'KNeighborsClassifier', 'num_trans': 'StandardScaler', 'cat_trans': 'OneHotEncoder'}
Score: 81.16%, Time: 0.1355 s
{'model': 'KNeighborsClassifier', 'num_trans': 'MinMaxScaler', 'cat_trans': 'OrdinalEncoder'}
Score: 79.89%, Time: 0.6737 s
{'model': 'KNeighborsClassifier', 'num_trans': 'MinMaxScaler', 'cat_trans': 'OneHotEncoder'}
Score: 80.05%, Time: 0.1117 s
{'model': 'KNeighborsClassifier', 'num_trans': 'Normalizer', 'cat_trans': 'OrdinalEncoder'}
Score: 80.72%, Time: 0.6573 s
{'model': 'KNeighborsClassifier', 'num_trans': 'Normalizer', 'cat_trans': 'OneHotEncoder'}
Score: 80.99%, Time: 0.1097 s
{'model': 'ExtraTreeClassifier', 'num_trans': 'StandardScaler', 'cat_trans': 'OrdinalEncoder'}
Score: 79.89%, Ti

Teraz w jednej tabeli mamy wszystkie interesujące dane, które mogą posłużyć nam chociażby do znalezienia najlepszego modelu:

In [43]:
models_df.sort_values('score', ascending=False)

,model,num_trans,cat_trans,score,time_elapsed
0,CatBoostClassifier,MinMaxScaler,OrdinalEncoder,0.871478,54.939374
0,CatBoostClassifier,StandardScaler,OrdinalEncoder,0.871478,83.523171
0,CatBoostClassifier,StandardScaler,OneHotEncoder,0.871367,54.575908
0,CatBoostClassifier,MinMaxScaler,OneHotEncoder,0.871367,50.578351
0,XGBClassifier,StandardScaler,OneHotEncoder,0.871035,59.816833
0,LGBMClassifier,MinMaxScaler,OneHotEncoder,0.870925,21.554955
0,LGBMClassifier,StandardScaler,OneHotEncoder,0.870925,1.521320
0,XGBClassifier,StandardScaler,OrdinalEncoder,0.870262,65.858596
0,XGBClassifier,MinMaxScaler,OrdinalEncoder,0.870262,5.584308
0,XGBClassifier,MinMaxScaler,OneHotEncoder,0.869267,11.546939


Ale *najlepszy* może być w różnych kategoriach - nie tylko skuteczności, ale też na przykład czasu uczenia czy też stabilności wyniku. Zobaczmy podstawowe statystyki dla typów modeli:

In [ ]:
models_df[['model', 'score', 'time_elapsed']] \
    .groupby('model') \
    .aggregate({
        'score': ['mean','std', 'min', 'max'],
        'time_elapsed': ['mean','std', 'min', 'max']
        }) \
    .reset_index() \
    .sort_values(('score', 'mean'), ascending=False)

Tutaj tak na prawdę nie mierzymy stabilności modelu - podajemy różnie przetworzone dane do tego samego modelu. Stablilność można zmierzyć puszczając na model fragmentaryczne dane, co można zautomatyzować porpzez *KFold*/*RepeatedKFold* (z sklearn.model_selection), ale dzisiaj nie o tym.

Sprawdźmy który rodzaj modelu daje najlepszą skuteczność:

In [ ]:
sns.boxplot(data=models_df, x='score', y='model')

Ostatnie trzy (XGBoost, LigthGBM i CatBoost) dają najlepsze wyniki i pewnie warto je brać pod uwagę w przyszłości.

A czy są różnice pomiędzy transformatorami?

In [ ]:
sns.boxplot(data=models_df, x='score', y='num_trans')

In [ ]:
sns.boxplot(data=models_df, x='score', y='cat_trans')

Przy tych danych wygląda, że właściwie nie ma większej różnicy (nie bijemy się tutaj o 0.01 punktu procentowego poprawy *accuracy* modelu). Może więc czas treningu jest istotny?

In [ ]:
sns.boxplot(data=models_df, x='time_elapsed', y='model')

Mamy kilku liderów, ale z tych które dawały najlepsze wyniki warto wziąć pod uwagę XGBoosta i LightGMB.

Dzięki przećwiczeniu kilku modeli mamy dwóch najbardziej efektywnych (czasowo) i efektownych (z najlepszym *accuracy*) kandydatów do dalszych prac. Wyszukanie ich to kilka linii kodu. Jeśli przyjdzie nam do głowy nowy model - dodajemy go do listy `classifiers`. Jeśli znajdziemy inny transformator - dopisujemy do listy `scalers` lub `cat_transformers`. Nie trzeba kopiować dużych kawałków kodu, nie trzeba właściwie pisać nowego kodu.

Dokładnie tym samym sposobem możemy poszukać *hyperparametrów* dla konkretnego modelu i zestawu transformacji w **pipeline**. Ale to już w następnym odcinku.

In [ ]:
# odpowiednio zmieniamy jego paramety - dobieramy transformatory
pipe_params = {
    'preprocessor__numerical__num_trans': StandardScaler(),
    'preprocessor__categorical__cat_trans': OneHotEncoder(),
    'classifier': LGBMClassifier()
}

pipe.set_params(**pipe_params)

In [ ]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, RepeatedStratifiedKFold
import numpy as np

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10)

In [ ]:
scores = cross_val_score(pipe, X_train, y_train, cv=cv)

In [ ]:
print(f"Mean: {scores.mean()}, median: {np.median(scores)}, std: {scores.std()}")

In [ ]:
print(100*scores.std()/scores.mean())

In [ ]:
sns.violinplot(scores)

In [ ]:
sns.scatterplot(x=list(range(0, len(scores))), y=scores)